In [6]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('2025_LoL_esports_match_data_from_OraclesElixir.csv')
print(f"Shape: {data.shape}")


Shape: (82908, 164)


C:\Users\Asus\AppData\Local\Temp\ipykernel_13232\2730736923.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2025_LoL_esports_match_data_from_OraclesElixir.csv')


In [27]:
# Criando lógica de agregação por match up

metadata_cols = ['gameid', 'position', 'champion', 'year', 'league', 'playoffs', 'split', 'side','result']

metric_cols = [
    'golddiffat15', 'xpdiffat15', 'csdiffat15',
    'killsat15', 'assistsat15', 'deathsat15',
    'opp_killsat15', 'opp_assistsat15', 'opp_deathsat15',
    'golddiffat25', 'xpdiffat25', 'csdiffat25',
    'killsat25', 'assistsat25', 'deathsat25',
    'opp_killsat25', 'opp_assistsat25', 'opp_deathsat25'
] # métricas a serem calculadas

# Filtrar apenas colunas relevantes
base = data[metadata_cols + metric_cols]

# Self-join para identificar o matchup dentro do mesmo jogo e mesma posição
# Similar a Cross Join do SQL

merged = base.merge(
    base,
    on=['gameid', 'position'],
    suffixes=('', '_matchup')
)

# Remover linhas onde o campeão é ele mesmo
merged = merged[merged['champion'] != merged['champion_matchup']]

# Agrupar e calcular médias
grouped = (
    merged.groupby([
        'year', 'league', 'playoffs', 'split',
        'side', 'position', 'champion', 'champion_matchup'
    ])
    [metric_cols]
    .mean()
    .reset_index()
    .fillna(0)
)

print(grouped.head())

games_played = (
    merged.groupby([
        'year', 'league', 'playoffs', 'split',
        'side', 'position', 'champion', 'champion_matchup'
    ])['gameid']
    .nunique()
    .reset_index()
    .rename(columns={'gameid': 'games_played'})
)

grouped = grouped.merge(games_played, on=[
    'year', 'league', 'playoffs', 'split',
    'side', 'position', 'champion', 'champion_matchup'
])

wins = merged[merged['result'] == 1]

victories = (
    wins.groupby([
        'year', 'league', 'playoffs', 'split',
        'side', 'position', 'champion', 'champion_matchup'
    ])['gameid']
    .nunique()
    .reset_index()
    .rename(columns={'gameid': 'wins'})
)

grouped = grouped.merge(victories, on=[
    'year', 'league', 'playoffs', 'split',
    'side', 'position', 'champion', 'champion_matchup'
], how='left')
grouped['wins'] = grouped['wins'].fillna(0).astype(int)

   year league  playoffs   split  side position  champion champion_matchup  \
0  2025     AL         0  Spring  Blue      bot  Aphelios           Ezreal   
1  2025     AL         0  Spring  Blue      bot      Ashe            Varus   
2  2025     AL         0  Spring  Blue      bot   Caitlyn            Ziggs   
3  2025     AL         0  Spring  Blue      bot     Corki     Miss Fortune   
4  2025     AL         0  Spring  Blue      bot     Corki            Varus   

   golddiffat15  xpdiffat15  ...  opp_deathsat15  golddiffat25  xpdiffat25  \
0        -448.0     -1059.0  ...             0.0       -2418.0     -2964.0   
1         -76.0      1013.0  ...             2.0         922.0       214.0   
2        -951.0       -67.0  ...             0.0       -1229.0       216.0   
3        1280.0       741.0  ...             1.0           0.0         0.0   
4        -111.5      -585.0  ...             1.0        -661.0     -1117.5   

   csdiffat25  killsat25  assistsat25  deathsat25  opp_killsat

In [ ]:
# Gerar arquivo para validação
grouped.to_excel('matchup_stats.xlsx', index=False)